# Introduzione
In questo file troviamo un modello di rete neurale per la classificazione del dataset COVID-CXR4.\
IL dataset è stato scaricato da [Kaggle](https://www.kaggle.com/datasets/andyczhao/covidx-cxr2) e messo dentro la cartella datasets \(questa cartella è ignorata da git perchè il dataset è grosso)\
I modelli salvati si possono trovare sotto la cartella [models](models) in modo da poterli usare senza rifare l'addestramento.

Questo *interactive pyhon notebook* è suddiviso in 3 parti principali:
- **Dataset**: in cui viene caricato, modificato e salvato in una cache l'intero dataset di immagini.
- **Modello**: in cui vengono creati e addestrati l'autoencoder e il classificatore.
- **Contrastive Learning**: in cui viene applicata la tecnica di contrastive learning per migliorare gli embedding da passare al classificatore.

Ogni parte del notebook contiene anche dei grafici e immagini per mostrare come i vari modelli si comportano.

In questa prima parte vengono importati le varie librerie usate e vengono create le variabili globali.

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras import layers, models, optimizers, ops, backend
from keras.api.preprocessing.image import load_img, img_to_array
from sklearn.metrics import ConfusionMatrixDisplay, f1_score, balanced_accuracy_score

upperdir = '..' # change this if the script is not in the same directory as the parent folder
models_dir = f"{upperdir}/models"
datasets_dir = f"{upperdir}/datasets"

# Ensure the directories exist
os.makedirs(models_dir, exist_ok=True)
os.makedirs(datasets_dir, exist_ok=True)

# Variabili per i modelli
shape = (224, 224)
latent_space = 64
paths = {
    'autoencoder': f"{models_dir}/autoencoder_{latent_space}_{shape[0]}x{shape[1]}.keras",
    'classifier': f"{models_dir}/classifier_{latent_space}_{shape[0]}x{shape[1]}.keras",
    'correction': f"{models_dir}/correction_{latent_space}_{shape[0]}x{shape[1]}.keras",
    'history_autoencoder': f"{models_dir}/history_auto_{latent_space}_{shape[0]}x{shape[1]}.npy",
    'history_classifier': f"{models_dir}/history_clf_{latent_space}_{shape[0]}x{shape[1]}.npy",
    'history_correction': f"{models_dir}/history_cor_{latent_space}_{shape[0]}x{shape[1]}.npy"
}

# Dataset
Modifica e caricamento del dataset.
Il dataset usato in questo caso è il dataset [COVIDx CXR-4](https://www.kaggle.com/datasets/andyczhao/covidx-cxr2).

Le modifiche apportate sono:
- Rimozione dei canali di colore \(alcune immagini hanno per esempio delle scritte rosse\); quindi ogni immagine è in scala di grigio.
- Ridimensionamento a 224x224 \(molte immagini sono 1024x1024 ma ci sono anche di dimensioni diverse\)

Le immagini importate sono sottoforma di array di numpy a 8bit che poi vengono salvate in un file cache (~4GB).\
Il primo blocco di codice dichiara delle funzioni utili per la modifica del dataset. La funzione `covid_cxr_data` è quella responsabile per il caricamento dei dati.

In [ ]:
def load_cache(file) -> dict:
    try:
        return dict(np.load(file, allow_pickle=True).item())
    except Exception:
        return {}

def images_to_numpy(images: pd.Series, size: tuple[int, int]):
    np_images = np.zeros((len(images), *size, 1), dtype=np.uint8)
    for i, img_name in enumerate(tqdm(images)):
        img = load_img(img_name, target_size=size, color_mode='grayscale')
        np_images[i] = img_to_array(img, dtype=np.uint8).reshape((*size, 1))
    return np_images

def classes_to_numpy(classes: pd.Series):
    return np.array(pd.factorize(classes)[0])

def covid_cxr_data(size:tuple[int, int]=(256, 256)):
    directory = f"{datasets_dir}/covid_cxr"
    cache = f"{directory}/cache_{size[0]}x{size[1]}.npy"
    dataset = load_cache(cache)

    if len(dataset) == 0:
        types = ['train', 'val', 'test']
        all_files = []
        for t in types:
            df = pd.read_csv(f"{directory}/{t}.txt", delimiter=' ', header=None)
            df[1] = df[1].apply(lambda x: f"{directory}/{t}/{x}")
            all_files.append(df)

        df = pd.concat(all_files)
        df.columns = ['id', 'filename', 'class', 'source']
        images = images_to_numpy(df['filename'], size)
        predictions = classes_to_numpy(df['class'])

        train_tot = len(all_files[0])
        val_tot = train_tot + len(all_files[1])
        test_tot = val_tot + len(all_files[2])

        dataset['train'] = (images[:train_tot], predictions[:train_tot])
        dataset['val'] = (images[train_tot:val_tot], predictions[train_tot:val_tot])
        dataset['test'] = (images[val_tot:test_tot], predictions[val_tot:test_tot])

        np.save(cache, dataset)

    x_train, y_train = dataset['train'][0], dataset['train'][1]
    x_val, y_val = dataset['val'][0], dataset['val'][1]
    x_test, y_test = dataset['test'][0], dataset['test'][1]
    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

def data_generator(x, batch_size):
    num_samples = x.shape[0]
    indices = np.arange(num_samples)
    while True:
        np.random.shuffle(indices)  # Mescola gli indici all'inizio di ogni epoca
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            batch = image_int_to_float(x[batch_indices])
            yield (batch, )

def data_generator_autoencoder(x, batch_size):
    for batch in data_generator(x, batch_size):
        yield (batch[0], batch[0])

def image_int_to_float(image:np.ndarray):
    return image.astype(np.float32) / 255.0


Di seguito carichiamo il dataset usando le funzioni dichiarate precedentemente e mostriamo quanti dati sono stati caricati per ogni tipologia \(training, validation, test\).

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = covid_cxr_data(shape)

print(f"Train: {x_train.shape}, {y_train.shape}")
print(f"Validation: {x_val.shape}, {y_val.shape}")
print(f"Test: {x_test.shape}, {y_test.shape}")

Di seguito viene mostrato quante classi ci sono e come sono distribuite all'interno del dataset.\
Come si può notare il training set è sbilanciato verso una classe e questo non aiuta per l'addestramento del classificatore.

In [ ]:
# Count the number of samples per class for each dataset
train_counts = np.bincount(y_train)
val_counts = np.bincount(y_val)
test_counts = np.bincount(y_test)

total_classes = len(train_counts)
class_weights = {i: len(y_train) / (total_classes * count) for i, count in enumerate(train_counts)}
total_weight = sum(class_weights.values())
class_weights = {k: v / total_weight for k, v in class_weights.items()}

# Plot the counts
plt.figure()
x_labels = range(total_classes)
plt.bar(x_labels, train_counts, width=0.2, label='Train', align='center')
plt.bar([x + 0.25 for x in x_labels], val_counts, width=0.2, label='Validation', align='center')
plt.bar([x + 0.5 for x in x_labels], test_counts, width=0.2, label='Test', align='center')
plt.xticks(x_labels, [f"Class {i}" for i in x_labels])
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('Number of Samples per Class in Each Dataset')
plt.legend()
plt.tight_layout()
plt.show()

# Modello
In questa sezione vediamo i modelli per l'autoencoding e per la classificazione.\
Come già illustrato precedentemente il processo dei modelli è il seguente:
- **Autoencoder** che prende in input delle immagini di grandezza definita nella prima parte di codice (shape), per comprimerle tramite layer convoluzionali fino ad una rappresentazione latente definita anch'essa in testa al codice (latent_space).\
  L'autoencoder a quel punto cercherà di ricostruire l'immagine dalla sua rappresentazione latente tramite la parte di decoder.\
  Il modello verrà valutato tramite *MSE* dato che i valori dei pixel saranno compresi tra \[0,1\]
- **Classifier** che prende in input la rappresentazione latente dell'immagine e la trasforma in probabilità di una o dell'altra classe.\
  Questo modello verrà valutato tramite *sparse_categorical_crossentropy* dato che restituirà un array con le probabilità per ogni classe.

### Autoencoder
Il primo modello creato è l'autoencoder e usa gli stessi principi delle CNN per creare una rappresentazione compatta delle immagini. Infatti il modello è composto da dei Convolutional Layer che, riducono la dimensione spaziale per aumentare la dimensionde dei filtri.\
L'encoder ha inoltre dei layer di BatchNormalization.

Questo modello è quello più lungo da addestrare solamente perchè ha abbastanza parametri e il dataset, essendo grande, non ci sta in memoria.\
Per queste ragioni la batch è piccola e generata da una funzione.

In [ ]:
try:
    autoencoder = models.load_model(paths['autoencoder'])
    encoder = autoencoder.get_layer('encoder')
    decoder = autoencoder.get_layer('decoder')

    enc_space = encoder.output_shape[1]
    if enc_space != latent_space:
        print(f"Encoder latent space mismatch: {enc_space} != {latent_space}")
    latent_space = enc_space

except Exception as e:
    in_encoder = layers.Input(shape=(*shape, 1))
    x = layers.BatchNormalization()(in_encoder)
    x = layers.Conv2D(32, 3, padding='same', strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(64, 3, padding='same', strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(128, 3, padding='same', strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(256, 3, padding='same', strides=2)(x)
    before_flatten = x.shape[1:]
    x = layers.Flatten()(x)
    flatten = x.shape[1]
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    latent = layers.Dense(latent_space, activation='sigmoid')(x)
    encoder = models.Model(in_encoder, latent, name='encoder')

    in_decoder = layers.Input(shape=(latent_space,))
    x = layers.Dense(flatten)(in_decoder)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape(before_flatten)(x)
    x = layers.Conv2DTranspose(256, 3, padding='same', strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2DTranspose(128, 3, padding='same', strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2DTranspose(64, 3, padding='same', strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2DTranspose(32, 3, padding='same', strides=2)(x)
    x = layers.LeakyReLU()(x)
    out_decoder = layers.Conv2DTranspose(1, 3, padding='same', activation='sigmoid')(x)
    decoder = models.Model(in_decoder, out_decoder, name='decoder')

    in_autoencoder = layers.Input(shape=(*shape, 1))
    encoder_out = encoder(in_autoencoder)
    decoder_out = decoder(encoder_out)
    autoencoder = models.Model(in_autoencoder, decoder_out, name='autoencoder')
    autoencoder.compile(optimizer=optimizers.Adam(), loss='mse')

    batch = 32
    epochs = 10
    batch_steps = len(x_train) // batch
    batch_val_steps = len(x_val) // batch
    gen_train = data_generator_autoencoder(x_train, batch)
    gen_val = data_generator_autoencoder(x_val, batch)

    history_auto = autoencoder.fit(gen_train, validation_data=gen_val,
                                   epochs=epochs, steps_per_epoch=batch_steps, validation_steps=batch_val_steps)

    autoencoder.save(paths['autoencoder'])
    np.save(paths['history_autoencoder'], history_auto.history)

Dopo aver trainato o caricato l'autoencoder facciamo la prediction dell'intero dataset:

In [ ]:
batch_predictions = 32

batch_train_steps = len(x_train) // batch_predictions + 1
batch_val_steps = len(x_val) // batch_predictions + 1
batch_test_steps = len(x_test) // batch_predictions + 1

gen_train = data_generator(x_train, batch_predictions)
gen_val = data_generator(x_val, batch_predictions)
gen_test = data_generator(x_test, batch_predictions)

x_train_encoded = encoder.predict(gen_train, steps=batch_train_steps)
x_val_encoded = encoder.predict(gen_val, steps=batch_val_steps)
x_test_encoded = encoder.predict(gen_test, steps=batch_test_steps)

assert x_train_encoded.shape[0] == len(x_train), f"{x_train_encoded.shape[0]} {len(x_train)}"
assert x_val_encoded.shape[0] == len(x_val), f"{x_val_encoded.shape[0]} {len(x_val)}"
assert x_test_encoded.shape[0] == len(x_test), f"{x_test_encoded.shape[0]} {len(x_test)}"

### Classificatore
Il classificatore è un modello semplice con 2 layer densi e un layer finale per la classificazione con la softmax.\
Essendo i dati molto più piccoli le batch possono essere alte e si possono avere molte più epoche per far imparare.

Purtroppo essendo il dataset molto sbilanciato verso una classe l'addestramento viene influenzato negativamente se non si fanno delle correzioni.

In [ ]:
try:
    classifier = models.load_model(paths['classifier'])

except Exception:
    in_classifier = layers.Input(shape=(latent_space,))
    x = layers.BatchNormalization()(in_classifier)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(32, use_bias=False)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(8, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    out_classifier = layers.Dense(total_classes, activation='softmax')(x)
    classifier = models.Model(in_classifier, out_classifier, name='classifier')
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    batch = 1024
    epochs = 10

    batch_steps = len(x_train) // batch
    batch_val_steps = len(x_val) // batch

    history_class = classifier.fit(x_train_encoded, y_train, validation_data=(x_val_encoded, y_val),
                                   epochs=epochs, batch_size=batch, class_weight=class_weights)

    classifier.save(paths['classifier'])
    np.save(paths['history_classifier'], history_class.history)

Anche dopo il classificatore, che sia stato caricato o addestrato dobbiamo predire i risultati e salvarli in variabili:

In [ ]:
batch_predictions = 32

y_train_pred = classifier.predict(x_train_encoded)
y_val_pred = classifier.predict(x_val_encoded)
y_test_pred = classifier.predict(x_test_encoded)

### Risultati
Di seguito i risultati dell'addestramento se è stato fatto, altrimenti vengono mostrati solo delle predizioni di alcuni dati di test.

In [ ]:
history = [
    np.load(paths['history_autoencoder'], allow_pickle=True).item(),
    np.load(paths['history_classifier'], allow_pickle=True).item()
]

plt.figure(figsize=(12, 5))
for i, h in enumerate(history):
    metric = list(h.keys())[0]
    plt.subplot(1, 2, i+1)
    plt.plot(h[metric], label=f'Training {metric}')
    plt.plot(h[f'val_{metric}'], label=f'Validation {metric}')
    plt.title(f'Model {metric}')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def plot_confusion_matrix(datasets:list[tuple[str, np.ndarray, np.ndarray]], figsize=None):
    fig, axes = plt.subplots(1, len(datasets), figsize=figsize)
    for i, (title, y_true, y_pred) in enumerate(datasets):
        y_pred = np.argmax(y_pred, axis=1)
        ConfusionMatrixDisplay.from_predictions(
            y_true,
            y_pred,
            normalize='true',
            display_labels=[i for i in range(total_classes)],
            cmap=plt.cm.Blues,
            colorbar=False,
            ax=axes[i]
        )
        axes[i].set_title(f"{title}\n"
                          + f"Accuracy: {balanced_accuracy_score(y_true, y_pred):.2%}\n"
                          + f"F1-Score: {f1_score(y_true, y_pred):.2f}")

    plt.tight_layout()
    plt.show()

datasets = [
    ("Training Data", y_train, y_train_pred),
    ("Validation Data", y_val, y_val_pred),
    ("Test Data", y_test, y_test_pred)
]
plot_confusion_matrix(datasets)


In [ ]:
# Choose N random images from the test set
total = 10
indices = np.random.choice(len(x_test), total, replace=False)
orig_img = x_test[indices]
orig_classes = y_test[indices]
pred_img = decoder.predict(x_test_encoded[indices], verbose=0)
pred_classes = y_test_pred[indices]

# Plot the original and predicted images
plt.figure(figsize=(15, 3.5))
for i, _ in enumerate(indices):
    ax = plt.subplot(2, total, i + 1)
    plt.imshow(orig_img[i], cmap='gray')
    plt.title(f"Original: {orig_classes[i]}")
    plt.axis('off')

    pred = np.argmax(pred_classes[i])
    correct = pred == orig_classes[i]

    ax = plt.subplot(2, total, i + total + 1)
    plt.imshow(pred_img[i], cmap='gray')
    plt.title(f"Pred: {pred_classes[i][1]:.2f}", color='green' if correct else 'red')
    plt.axis('off')
plt.show()

# Contrastive Learning
Il blocco di codice sottostante definisce un modello siamese che utilizza la loss contrastive per modificare gli embedding in modo da migliorare la classificazione.\
Il modello contrastivo è un semplice modello con due layer densi che produce un output scalare con attivazione sigmoid, rappresentando la probabilità che due input appartengano alla stessa classe.\
Il modello siamese utilizza due torri identiche del modello contrastivo per calcolare la distanza tra due rappresentazioni latenti, utilizzando una funzione Lambda personalizzata per calcolare la distanza.

In [ ]:
def make_pairs_generator(x, y_true, y_pred, batch):
    classes = len(np.unique(y_true))
    correct_predictions = np.where(y_true == y_pred)[0]
    correct_predictions_class = [np.where(y_true[correct_predictions] == i)[0] for i in range(classes)]

    not_correct = np.where(y_true != y_pred)[0]
    yield len(not_correct)

    while True:
        np.random.shuffle(not_correct)
        for i in not_correct:
            pairs_1 = []
            pairs_2 = []
            labels = []

            for _ in range(batch // 2):
                # Positive pair
                x1 = x[i]
                label1 = y_true[i]
                x2 = x[np.random.choice(correct_predictions_class[label1])]
                pairs_1 += [x1]
                pairs_2 += [x2]
                labels += [0]

                # Negative pair
                label2 = np.random.choice(classes)
                while label2 == label1:
                    label2 = np.random.choice(classes)
                idx = np.random.choice(correct_predictions_class[label2])
                x2 = x[idx]
                pairs_1 += [x1]
                pairs_2 += [x2]
                labels += [1]

            yield (np.array(pairs_1), np.array(pairs_2)), np.array(labels)

batch_train_pairs = 32
gen_train_pair = make_pairs_generator(x_train_encoded, y_train, np.argmax(y_train_pred, axis=1), batch_train_pairs)
gen_val_pair = make_pairs_generator(x_val_encoded, y_val, np.argmax(y_val_pred, axis=1), batch_train_pairs)
batch_train_pairs_steps = next(gen_train_pair)
batch_val_pairs_steps = next(gen_val_pair)

print(f"Train pairs: {batch_train_pairs}, step: {batch_train_pairs_steps}")
print(f"Validation pairs: {batch_train_pairs}, step: {batch_val_pairs_steps}")

Questo codice implementa due loss functions.\
In particolare per problemi di apprendimento contrastivo, come il confronto tra coppie di campioni vengono usate le seguenti loss:

1. contrastive_loss: calcola la perdita contrastiva standard.\
   Penalizza le coppie di campioni in base alla loro distanza predetta (y_pred) e alla loro etichetta reale (y_true).\
   Se i campioni sono simili (y_true=0), penalizza le distanze grandi.\
   Se i campioni sono diversi (y_true=1), penalizza le distanze piccole.
2. contrastive_SNN_loss: calcola una variante della perdita contrastiva basata su una funzione softmax normalizzata.\
   Utilizza una temperatura (temperature) per controllare la "morbidezza" della penalizzazione.\
   Penalizza le coppie in base alla probabilità relativa di similarità, calcolata come rapporto tra le distanze esponenziali normalizzate.\
   La loss usata è la [Soft Nearest Neighbors Loss](https://lilianweng.github.io/posts/2021-05-31-contrastive/#soft-nearest-neighbors-loss)

In [ ]:
temperature = 1

def distance(vects):
    x, y = vects
    sum_square = ops.sum(ops.square(x - y), axis=1, keepdims=True)
    return ops.sqrt(ops.maximum(sum_square, backend.epsilon()))

def loss(margin=1.0, temperature=1.0):
    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        square_pred = ops.square(y_pred)
        margin_square = ops.square(ops.maximum(margin - (y_pred), 0))
        return ops.mean((1 - y_true) * square_pred + (y_true) * margin_square)

    def contrastive_SNN_loss(y_true, y_pred):
        mask = ops.equal(y_true, 0)
        exp_similarity = ops.exp(ops.negative(y_pred / temperature))

        numerator = ops.sum(exp_similarity * mask)
        denominator = ops.sum(exp_similarity) + backend.epsilon()  # Add epsilon to avoid division by zero

        safe_ratio = numerator / denominator
        safe_ratio = ops.maximum(safe_ratio, backend.epsilon())  # Ensure ratio is not less than epsilon

        return ops.negative(ops.mean(ops.log(safe_ratio)))

    return contrastive_SNN_loss


Il modello e la rete siamese usata per l'addestramento sono i seguenti:

In [ ]:
half_space = int(latent_space/2)

correction_in = layers.Input(shape=(latent_space,))
x = layers.Dense(half_space, activation='relu')(correction_in)
correction_out = layers.Dense(latent_space, activation='sigmoid')(x)
correction = models.Model(correction_in, correction_out, name='correction')

siamese_in_1 = layers.Input(shape=(latent_space,))
siamese_in_2 = layers.Input(shape=(latent_space,))
siamese_tower_1 = correction(siamese_in_1)
siamese_tower_2 = correction(siamese_in_2)
x = layers.Lambda(distance, output_shape=(1,))([siamese_tower_1, siamese_tower_2])
x = layers.BatchNormalization()(x)
siamese_out = layers.Dense(1, activation="sigmoid")(x)
siamese = models.Model([siamese_in_1, siamese_in_2], siamese_out, name='siamese')
siamese.compile(optimizer=optimizers.RMSprop(), loss=loss(), metrics=['accuracy'])

history_siamese = siamese.fit(gen_train_pair, validation_data=gen_val_pair,
                              epochs=10, batch_size=batch_train_pairs,
                              steps_per_epoch=batch_train_pairs_steps, validation_steps=batch_val_pairs_steps)
correction.save(paths['correction'])
np.save(paths['history_correction'], history_siamese.history)

### Risultati


In [ ]:
from sklearn.decomposition import PCA as dim_reduction

# Get sample batch
indices = np.random.choice(len(x_test), 1000, replace=False)
embedded_true = x_test_encoded[indices]
embedded_new = correction.predict(embedded_true, verbose=0)
labels_true = y_test[indices]
labels_pred = np.argmax(y_test_pred[indices], axis=1)
labels_new = np.argmax(classifier.predict(embedded_new, verbose=0), axis=1)

# Get low-dimensional t-SNE Embeddings
h_embedded_true = dim_reduction(n_components=2).fit_transform(embedded_true)
h_embedded_new = dim_reduction(n_components=2).fit_transform(embedded_new)

# Plot
colors = list(plt.cm.tab10.colors[:total_classes])
colors_true = [colors[i] for i in labels_true]
colors_pred = [colors[i] for i in labels_pred]
colors_new = [colors[i] for i in labels_new]

plots = [
    (h_embedded_true, colors_true, 'Original Space'),
    (h_embedded_new, colors_true, 'New Space'),
    (h_embedded_true, colors_pred, 'Predicted Space'),
    (h_embedded_new, colors_new, 'Predicted New Space')
]

plt.figure(figsize=(12, 6))
for i, (h, colors, title) in enumerate(plots):
    plt.subplot(2, 2, i + 1)
    plt.scatter(h[:,0], h[:,1], alpha=0.5, c=colors)
    plt.title(title)
plt.show()

In [ ]:
plot_confusion_matrix([
    ("Training Data", y_train, y_train_pred),
    ("Training Data (Corrected)", y_train, classifier.predict(correction.predict(x_train_encoded, verbose=0), verbose=0))
])
plot_confusion_matrix([
    ("Validation Data", y_val, y_val_pred),
    ("Validation Data (Corrected)", y_val, classifier.predict(correction.predict(x_val_encoded, verbose=0), verbose=0))
])
plot_confusion_matrix([
    ("Test Data", y_test, y_test_pred),
    ("Test Data (Corrected)", y_test, classifier.predict(correction.predict(x_test_encoded, verbose=0), verbose=0))
])